# Ex08 Aplicações da DFT

## Parte 1 - Filtragem no Domínio da Frequência

Dos itens abaixo *a*, *b* e *c* você deve fazer obrigatoriamente o item *a* e no mínimo mais um item: *b* ou *c*. Nada impede você de fazer os 3 itens: *a*, *b* e *c*.

### a. Projetando filtros no domínio da frequência

Para projetar os filtros no domínio da frequência, utilize imagens sintéticas, como círculo ou quadrado ou retângulo (filtros ideais), tomando-se o cuidado para verificar se estes filtros são complexos-conjugados. Se preciso, crie uma função que retorne *True*, caso a imagem seja complexa conjugada e *False*, caso contrário. Lembre-se também que o projeto do filtro é normalmente feito no espectro ótico de Fourier, mas sua aplicação é feita com coordenadas 0 a N-1.

Teste os filtros projetados filtrando alguma imagem.

### b. Filtro Butterworth

Crie uma função para projetar um filtro passa-baixas Butterworth. A função de transferência do filtro passa-baixas de Butterworth de ordem $n$ e com posição da frequência de corte a uma distância $D_0$ da origem é definida pela relação $$ H(u,v) = \frac{1}{1 + [\frac{D(u,v)}{D_0}]^{2n}}, $$ onde $n$ é a ordem do filtro. Para facilitar a implementação, podemos usar a seguinte expressão: $$ H(u,v) = \frac{1}{1 + (\sqrt{2} - 1)(\sqrt{(\frac{u}{N})^2 + (\frac{v}{M})^2)}.t_c)^{2n}}$$ com $$ u \in{[-\frac{N}{2},N - \frac{N}{2} -1]}$$ $$ v \in{[-\frac{M}{2},M - \frac{M}{2} -1]}$$ $$ t_c \in{[2, max\{N,M\}]}$$

Compare o resultado da filtragem de uma imagem usando um filtro ideal e o filtro de Butterworth.

### c.  Filtrando uma imagem com textura

Veja que a imagem do código de barras a seguir possui uma textura no fundo. Projete um filtro (em frequencia) que elimine esta textura, sem borrar demais a imagem.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

f = mpimg.imread('../data/barcode.tif')
plt.figure(figsize=(8,8))
plt.imshow(f,cmap='gray');
plt.show()

In [ ]:
import numpy as np
from numpy.fft import fft2,ifft2
import sys,os
ia898path = os.path.abspath('../../')
if ia898path not in sys.path:
    sys.path.append(ia898path)
import ia898.src as ia

In [ ]:
def shift_slice(img, rr, cc):
    
    f = np.array(img)
    r, c = f.shape

    c1 = cc%c
    r1 = rr%r
    g = np.concatenate(( np.concatenate((f[r-r1:,c-c1:], f[r-r1:,:c-c1]), axis=1) , \
                         np.concatenate((f[:r-r1, c-c1:], f[:r-r1, :c-c1]), axis=1) ))
    return g

### Parte 1.a

In [ ]:
f1_p1 = np.array(mpimg.imread('../data/gull.pgm'))
W1, H1 = f1_p1.shape

In [ ]:
H3_p1 = ia.circle(f1_p1.shape, 50, [W1//2, H1//2])
H3_p1_shift = shift_slice(H3_p1, H3_p1.shape[0]//2, H3_p1.shape[1]//2)
H2_p1 = np.ones(f1_p1.shape)
H2_p1[(W1//2)-50:-((W1//2)-49),(H1//2)-50:-((H1//2)-49)] = 0
H2_p1_shift = shift_slice(H2_p1, H2_p1.shape[0]//2, H2_p1.shape[1]//2)

fig = plt.figure(figsize=[12, 10])

fig.add_subplot(221)
plt.imshow(H3_p1, cmap='gray')

fig.add_subplot(222)
plt.imshow(H3_p1_shift, cmap='gray')

fig.add_subplot(223)
plt.imshow(H2_p1, cmap='gray')

fig.add_subplot(224)
plt.imshow(H2_p1_shift, cmap='gray')
plt.show()

In [ ]:
def show_fft(F):
    return shift_slice(np.log(np.abs(F)+1), F.shape[0]//2, F.shape[1]//2)

In [ ]:
F1_p1 = np.fft.fft2(f1_p1)
G4_p1 = F1_p1 * H2_p1_shift
g4_p1 = np.fft.ifft2(G4_p1)

G5_p1 = F1_p1 * H3_p1_shift
g5_p1 = np.fft.ifft2(G5_p1)

In [ ]:
fig = plt.figure(figsize=[12, 15])

fig.add_subplot(321)
plt.imshow(f1_p1, cmap='gray')

fig.add_subplot(322)
plt.imshow(show_fft(F1_p1), cmap='gray')

fig.add_subplot(324)
plt.imshow(show_fft(G4_p1), cmap='gray')

fig.add_subplot(323)
plt.imshow(g4_p1.real, cmap='gray')

fig.add_subplot(325)
plt.imshow(g5_p1.real, cmap='gray')

fig.add_subplot(326)
plt.imshow(show_fft(G5_p1), cmap='gray')

plt.show()

### Parte 1.c

In [ ]:
H1_p1 = ia.circle([256, 256], 50, [128,128])
H1_p1_shift = shift_slice(H1_p1, H1_p1.shape[0]//2, H1_p1.shape[1]//2)

In [ ]:
# F_p1 = shift_slice( show_fourier(np.fft.fft2(f)), f.shape[0]//2, f.shape[1]//2 )
F_p1 = np.fft.fft2(f)

plt.imshow(np.log(np.abs(F_p1)+1), cmap='gray')
plt.show()

In [ ]:
G1_p1 = F_p1 * H1_p1_shift

plt.imshow(np.log(np.abs(G1_p1)+1), cmap='gray')
plt.show()

g1_p1 = np.fft.ifft2(G1_p1).real

In [ ]:
h_p1 = ia.histogram(ia.normalize(np.log(np.abs(F_p1) + 1)))
plt.plot(h_p1)
plt.show()

In [ ]:
T1_p1 = np.arange(256).astype('uint8')
T2_p1 = ia.normalize(T1_p1 > 130)
T3_p1 = ia.normalize(255 - T1_p1)
fig = plt.figure(figsize=[10,10])
g3_p1 = T2_p1[ia.normalize(np.log(np.abs(F_p1) + 1))]
g2_p1 = T3_p1[g3_p1]

H1_p1 = ia.circle([256, 256], 50, [128,128])

H1_p1[(H1_p1.shape[0]//2)-5:(H1_p1.shape[0]//2)+5, :]=255
H1_p1[:, (H1_p1.shape[0]//2)-5:(H1_p1.shape[0]//2)+5]=255

H1_p1_shift = shift_slice(H1_p1, H1_p1.shape[0]//2, H1_p1.shape[1]//2)

H_final = g2_p1+ia.normalize(H1_p1_shift)

fig = plt.figure(figsize=[12,10])
fig.add_subplot(221)
plt.imshow(g3_p1, cmap='gray')

fig.add_subplot(222)
plt.imshow(ia.normalize(H1_p1_shift)//2, cmap='gray')

fig.add_subplot(223)
plt.imshow(H_final, cmap='gray')

fig.add_subplot(224)
plt.imshow(g2_p1+ia.normalize(H1_p1_shift), cmap='gray')

plt.show()


In [ ]:
G_final = F_p1 * H_final

g_final = np.fft.ifft2(G_final).real

In [ ]:
fig = plt.figure(figsize=[20,30])
fig.add_subplot(321)
plt.imshow(f, cmap='gray')

fig.add_subplot(322)
plt.imshow(np.log(np.abs(F_p1)+1), cmap='gray')

fig.add_subplot(323)
plt.imshow(g1_p1, cmap='gray')

fig.add_subplot(324)
plt.imshow(np.log(np.abs(G1_p1)+1), cmap='gray')

fig.add_subplot(325)
plt.imshow(g_final, cmap='gray')

fig.add_subplot(326)
plt.imshow(np.log(np.abs(G_final) + 1), cmap='gray')

plt.show()

## Parte 2 - Identificando Translação/Rotação por Correlação de Fase

Dos itens abaixo *a*, *b* e *c* você deve fazer obrigatoriamente o item *a* e no mínimo mais um item: *b* ou *c*. Nada impede você de fazer os 3 itens: *a*, *b* e *c*.

Através da Correlação de fase é possível identificar uma translação ou uma rotação sofrida por uma imagem (veja o notebook [Correlação de Fase](13 Correlacao de fase.ipynb)). 

### a. Coordenada polar da imagem ou da DFT?

Para identificar a rotação, a imagem é transformada para coordenadas polares, para depois ser aplicada a Transformada de Fourier e então calculada a correlação de fase. Verifique se é equivalente fazer a transformada de Fourier e só depois fazer a conversão para coordenadas polares no domínio da frequência para então computar a correlação de fase;

### b. Rotação e translação simultaneas

Imagine agora que uma imagem tenha sofrido rotação e translação simultaneamente. Tente agora identificar ambas transformações com esta mesma técnica. (DICA: Tente resolver o problema em 2 etapas, ou seja, aplicando 2 vezes os passos para a correlação de fase);
    c. (Opcional) Identifique o quão robusta é esta técnica, com relação a: ruído, variação de contraste, escala

### c. *Template Matching* 

Experimente resolver um problema de *Template Matching* usando correlação fase. Ou seja, recorte um pedaço de uma imagem e tente encontrar este pedaço na imagem original maior.

In [ ]:
def phasecorr(f,h):
    F = np.fft.fft2(f)
    H = np.fft.fft2(h)
    T = F * np.conjugate(H)
    R = T/np.abs(T)
    g = np.fft.ifft2(R)
    return g.real

In [ ]:
f1_p2 = mpimg.imread('../data/gull.pgm')


# Inserindo uma borda de zeros para permitir a rotação da imagem
t = np.zeros(np.array(f1_p2.shape)+100,dtype=np.uint8)
t[50:f1_p2.shape[0]+50,50:f1_p2.shape[1]+50] = f1_p2
f1_p2 = t
    
t1 = np.array([
             [1,0,-f1_p2.shape[0]/2.],
             [0,1,-f1_p2.shape[1]/2.],
             [0,0,1]]);

t2 = np.array([
             [1,0,f1_p2.shape[0]/2.],
             [0,1,f1_p2.shape[1]/2.],
             [0,0,1]]);

plt.imshow(f1_p2, cmap='gray')
plt.show()

In [ ]:
# Rotacionando a imagem -30 graus
W,H = f1_p2.shape
theta = np.radians(60)
r1 = np.array([
        [np.cos(theta),-np.sin(theta),0],
        [np.sin(theta),np.cos(theta),0],
        [0,0,1]]);
    
T = t2.dot(r1).dot(t1)
f_rot = ia.affine(f1_p2,T,0)

plt.figure(1,(10,10))
plt.subplot(1,2,1)
plt.imshow(f1_p2, cmap='gray')
plt.title('Imagem original')

plt.subplot(1,2,2)
plt.imshow(f_rot, cmap='gray')
plt.title('Imagem rotacionada')
plt.show()

F1_p2 = np.fft.fft2(f1_p2)
F1_rot = np.fft.fft2(f_rot)

plt.figure(1,(10,10))
plt.subplot(1,2,1)
plt.imshow(shift_slice(np.log(np.abs(F1_p2) + 1), W//2,H//2), cmap='gray')
plt.title('Imagem original')

plt.subplot(1,2,2)
plt.imshow(shift_slice(np.log(np.abs(F1_rot) + 1), W//2,H//2), cmap='gray')
plt.title('Imagem rotacionada')
plt.show()

In [ ]:
F_polar = ia.polar(F1_p2.real,(150,200),2*np.pi)
F_rot_polar = ia.polar(F1_rot.real,(150,200),2*np.pi)

plt.figure(1,(10,10))
plt.subplot(1,2,1)
plt.imshow(F_polar, cmap='gray')
plt.title('Imagem original (coord. polar)')

plt.subplot(1,2,2)
plt.imshow(F_rot_polar, cmap='gray')
plt.title('Imagem rotacionada (coord. polar)')

plt.show()

In [ ]:
T = F_polar * np.conjugate(F_rot_polar)
R = T/np.abs(T)
g = np.fft.ifft2(R)
g.real

# Encontrando o ponto de máxima correlação 
i = np.argmax(g.real)
corr = np.unravel_index(i,g.real.shape)

# Calculate the angle
ang = (float(corr[1])/g.real.shape[1])*360
print('Ponto de máxima correlação: ',ang)

In [ ]:
W,H = f.shape
f_polar = ia.polar(f1_p2,(150,200),2*np.pi)
f_rot_polar = ia.polar(f_rot,(150,200),2*np.pi)

plt.figure(1,(10,10))
plt.subplot(1,2,1)
plt.imshow(f_polar, cmap='gray')
plt.title('Imagem original (coord. polar)')

plt.subplot(1,2,2)
plt.imshow(f_rot_polar, cmap='gray')
plt.title('Imagem rotacionada (coord. polar)')
# Calculando a correlação de fase
g = ia.phasecorr(f_polar,f_rot_polar)

# Encontrando o ponto de máxima correlação 
i = np.argmax(g)
corr = np.unravel_index(i,g.shape)

# Calculate the angle
ang = (float(corr[1])/g.shape[1])*360
print('Ponto de máxima correlação: ',ang)

### Parte 2.c
**Fernando:** Não sei se o método foi muito preciso ao ser utilizado para localizar um fragmento dentro de uma imagem. Na primeira imagem, da gaivota, a localização está totalmente fora do real.
Na imagem seguinte, da menina, pareceu funcionar, mas eu não mudei a localização do fragmento dentro da imagem.

In [ ]:
f2_p2 = np.array(mpimg.imread('../data/gull.pgm'))
f3_p2 = np.zeros(f2_p2.shape)
f3_p2[10:61,10:61] = f2_p2[100:151,100:151]
# f3_p2[100:151,100:151] = f2_p2[100:151,100:151]

g1_p2 = ia.phasecorr(f2_p2, f3_p2)

plt.imshow(f3_p2, cmap='gray')
plt.show()

i = np.argmax(g1_p2)
l,c = np.unravel_index(i, g1_p2.shape)
corr = np.array(f2_p2.shape) - np.array((l,c))
print(corr)

In [ ]:
f2_p2[corr[0]-5:corr[1]+5, corr[1]-5:corr[1]+5] = 0
plt.imshow(f2_p2, cmap='gray')
plt.show()

In [ ]:
f4_p2 = np.array(mpimg.imread('../data/lenina.pgm'))
f5_p2 = np.zeros(f4_p2.shape)
# f5_p2[10:61,10:61] = f4_p2[100:151,100:151]
f5_p2[100:151,100:151] = f4_p2[100:151,100:151]

g2_p2 = ia.phasecorr(f4_p2, f5_p2)

plt.imshow(f5_p2, cmap='gray')
plt.show()

i = np.argmax(g2_p2)
l,c = np.unravel_index(i, g2_p2.shape)
corr = np.array(f4_p2.shape) - np.array((l,c))
print(corr)

In [ ]:
f4_p2[corr[0]-5:corr[1]+5, corr[1]-5:corr[1]+5] = 0
plt.imshow(f4_p2, cmap='gray')
plt.show()